## Search Engine with HuggingFace

<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G/blob/main/days/w1d4/Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this tutorial is to understand the basics of using the most used library in modern NLP: Huggingface.

We will try to understand the first steps in creating a semantic search engine.

You can read a pandas cheat sheet before using it and read about the `.loc` and `.iloc` methods.

https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

## Download the list of papers

In [1]:
!wget https://github.com/EffiSciencesResearch/ML4G/raw/4010bb6ccd63dee5896b26ee3c045898e0cf9ed6/days/w1d4/keynesian_eco_ML4G.xlsx -q

In [2]:
%pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.0 MB/s eta 0:00:00


In [3]:
import pandas as pd

df = pd.read_excel("keynesian_eco_ML4G.xlsx")

In [4]:
df

,id,nbInCitations,nbOutCitations,authors,paperAbstract,title,year,doi,doiUrl,fieldsOfStudy,journalName,magId,s2Url
0,c2890ae1a86d104b0b5b2cf71d890cb514b6590b,0,0,"[['2069055537', 'Robin Boadway'], ['119227414...",NaN,Fiscal Federalism: Preface,2009.0,10.1017/CBO9780511626883.001,https://doi.org/10.1017/CBO9780511626883.001,['Economics'],NaN,1597197407,NaN
1,b849439367ccb03b11183b9e3fdf594aae90255a,12,0,"[['116983853', 'Nancy J. Wulwick']]",NaN,Two Econometric Replications: The Historic Phi...,1996.0,10.1215/00182702-28-3-391,https://doi.org/10.1215/00182702-28-3-391,['Economics'],History of Political Economy,2007001276,NaN
2,1cc40d9cc0ee8978339d55b0406cb88014710ae9,0,0,"[['40266255', 'Julia M. Colston'], ['46784895'...",NaN,An adenoviral model to unlock the secrets of m...,2013.0,10.1016/J.JINF.2013.07.009,https://doi.org/10.1016/J.JINF.2013.07.009,['Economics'],Journal of Infection,2064032475,NaN
3,13bdff11c2e33df661d1bd7f08f7ab0bfe46acb6,0,0,"[['119468193', 'Baldwin Ranson']]",NaN,The Keynesian Revolution and its Critics,1988.0,10.1080/00213624.1988.11504824,https://doi.org/10.1080/00213624.1988.11504824,['Economics'],NaN,2565556283,NaN
4,6286ee0c109ac7ce25a9d8cdc15281ed4d218cfa,4,1,"[['119242604', 'Josef Steindl']]",The control of the economy is examined in term...,The control of the economy,2013.0,10.1007/978-1-349-20821-0_16,https://doi.org/10.1007/978-1-349-20821-0_16,['Economics'],PSL Quarterly Review,1804776193,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15293,1a68ce621afac192a4b7c3c30152fb9f210ec0d2,0,0,"[['1883404', 'Christian Fries']]",NaN,Heath‐Jarrow‐Morton Framework: Foundations,2007.0,10.1002/9780470179789.CH22,https://doi.org/10.1002/9780470179789.CH22,['Economics'],NaN,1499640240,NaN
15294,c637aa731bd2423633e75067ae039ec6133e32eb,0,0,"[['47021855', 'Miriam Smith']]",NaN,"Discrimination, From Romer to Vriend, 1986–2000",2008.0,10.4324/9780203895016-9,https://doi.org/10.4324/9780203895016-9,['Economics'],NaN,3028705958,NaN
15295,f6375b906fc8496a361ef1c083624424af98ad7e,50,22,"[['2990724', 'Imad A. Moosa']]",Abstract Okun's coefficient is estimated from ...,"Cyclical output, cyclical unemployment, and Ok...",1999.0,10.1016/S1059-0560(99)00028-3,https://doi.org/10.1016/S1059-0560%2899%2900028-3,['Economics'],International Review of Economics & Finance,2037151286,NaN
15296,b493013938a9b18de3009164cc9c21a95acb662c,5,3,"[['90362779', 'João Sicsú']]",The article criticizes the main hypothesis of ...,A NEGAÇÃO DA INEFICÁCIA DA POLÍTICA MONETÁRIA:...,2009.0,10.22456/2176-5456.10546,https://doi.org/10.22456/2176-5456.10546,['Economics'],NaN,1957768946,NaN


## Semantic search engine

Create a search engine by embedding by using https://huggingface.co/sentence-transformers/all-mpnet-base-v2

Make some queries and check that it works.

Switch to the GPU mode in Colab!
NB: In the sentence-transformers library, `model.encode(sentences)` can process an arbitrary number of sentences by batching automatically. When using the transformer library, you generally have to batch yourself.

Notes :

- I used this https://huggingface.co/learn/nlp-course/chapter5/6?fw=tf#using-faiss-for-efficient-similarity-search


In [5]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.4 MB/s eta 0:00:00


In [9]:
query = "Poverty in the US"

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# # Check if CUDA is available ans switch to GPU
# if torch.cuda.is_available():
#    device = torch.device("cuda")
#    model = model.to(device)
# print(model.device)

def get_embeddings(texts):

    # Tokenize sentences
    encoded_input = tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt"
    )
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Perform pooling
    query_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    # Normalize embeddings
    query_embedding = F.normalize(query_embedding, p=2, dim=1)

    return query_embedding

query_embedding = get_embeddings(query)

print("Query embeddings:")
print(query_embedding.shape)

Query embeddings:
torch.Size([1, 768])


In [10]:
# Format Data
from datasets import Dataset

papers = df[["id","paperAbstract","title","fieldsOfStudy","journalName"]]
papers = papers.fillna("")
papers = papers[:10]
paper_dataset = Dataset.from_pandas(papers)
paper_dataset

Dataset({
    features: ['id', 'paperAbstract', 'title', 'fieldsOfStudy', 'journalName'],
    num_rows: 10
})

In [11]:
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["paperAbstract"]
        + " \n "
        + examples["fieldsOfStudy"]
        + " \n "
        + examples["journalName"]
    }

paper_dataset = paper_dataset.map(concatenate_text)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [12]:
# Notice that we’ve converted the embeddings to NumPy arrays
# That’s because 🤗 Datasets requires this format when we try to index them with FAISS, which we’ll do next.

embeddings_dataset = paper_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).numpy()[0]}
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [15]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
# !python -m pip install --upgrade faiss faiss-cpu
# !pip install faiss-cpu --no-cache
# !pip install faiss-gpu
# !pip install faiss-cpu
!pip install faiss

import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (1:10.0-50~exp1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [39]:
import numpy as np

# FAISS : Similarity
# embeddings_dataset.add_faiss_index(column="embeddings")

# embeddings = np.array([embedding for embedding in embeddings_dataset["embeddings"]]).astype("float32")

# # Step 2: Instantiate the index
# index = faiss.IndexFlatL2(len(embeddings_dataset["embeddings"]))

# # Step 3: Pass the index to IndexIDMap
# index = faiss.IndexIDMap(index)

# # Step 4: Add vectors and their IDs
# index.add_with_ids(embeddings, embeddings_dataset["id"])

ValueError: ignored

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", query_embedding.numpy(), k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print("=" * 50)
    print()

## Few Shot Learning: Tldr

Use GPT-J 6b with some prompt engineering to create tldr of the summaries

You can begin with: https://huggingface.co/gpt2


The first step is to create a single tldr. For that, you will use some  few-shot learning. You can use this link to create your prompt: https://github.com/EffiSciencesResearch/ML4G/blob/38f80110be0802837254c1cd888f387475c9b5fe/days/w1d4/tldr_dataset.csv


After having created a single tldr, the aim is to add a new column in the pandas dataframe containing the tldr:
- automate the process and make inferences by batch. Store the inferences in a new column in the dataframe.
- Use the tqdm library to create a progress bar.
- Notice that it is too slow and switch to GPU. To do this, use ".to(device)" on the output tensor of the tokenizer and on the model.
- Use the command 'nvidia-smi' in the terminal to monitor the GPU usage. Aim at a GPU usage percentage of 70%.
- How does the speed of inference vary with the batch size?
- How does the inference speed vary with the padding policy in the tokenizer?
- How does the quality/speed of inference vary with the beam_search parameter?
- Bonus: read https://huggingface.co/blog/how-to-generate
- Bonus: Use a bigger model https://huggingface.co/EleutherAI/gpt-j-6B


In [40]:
# No need to
!wget https://raw.githubusercontent.com/EffiSciencesResearch/ML4G/38f80110be0802837254c1cd888f387475c9b5fe/days/w1d4/tldr_dataset.csv -q
df_tldr = pd.read_csv("tldr_dataset.csv")

In [41]:
prompt = "You must craft a good prompt by selecting 2 good abstract-tldr pairs from from df_tldr"
# NB: An example of working prompt is written below.

from transformers import pipeline
# NB: Note that we will use gpt2 because colab may not work.
generator = pipeline('text-generation', model = 'gpt2')
generator(prompt, max_length = 30, num_return_sequences=3)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},
##  {'generated_text': "Hello, I'm a language modeler. I write and maintain software in Python. I love to code, and that includes coding things that require writing"}, ...

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You must craft a good prompt by selecting 2 good abstract-tldr pairs from from df_tldr. To do this, you can'},
 {'generated_text': 'You must craft a good prompt by selecting 2 good abstract-tldr pairs from from df_tldr.sh, with the first pair'},
 {'generated_text': 'You must craft a good prompt by selecting 2 good abstract-tldr pairs from from df_tldr and add this to your rcf'}]

## Quality filtering

(Bonus) Implement a strategy to keep only high quality tldr

## Fine-Tuning

Bonus: Fine Tune T5-base from the corpus generated by  GPT-J to accelerate the inference and fine tune your first LLM

Use: https://huggingface.co/docs/transformers/training#training-hyperparameters

(You can use either the Native Pytorch method of the Trainer object. Both are fine.)

In [ ]:
# Working Prompt to summarize some abstracts
prompt = """I'm a TL;DR text generator, and need to generate a corresponding TL;DR from the following abstracts.

Abstract : "Review(s) of: The House of Rothschild: The World's Banker, by Niall Ferguson, Two vols, Penguin, 2000."
TL;DR : "The House of Rothschild: The World's Banker."

Abstract : "The benefits and drawbacks of sub‐contracting distribution are reviewed. The financial aspects of the decision, whether sub‐contracting is to be wholly adopted or in part, are discussed together with the implications for management."
TL;DR : "Benefits and drawbacks of subcontracting distribution are reviewed."

Abstract : "A Research Project Report by Kyagera Benjamin, Submitted to the Chandaria School of Business in Partial Fulfillment of the Requirement for the Degree of Master of Business Administration"
TL;DR : "Chandaria School of Business in Partial Fulfillment of the Degree of Master of Business Administration."

Abstract : "AbstractPolitical science and public policy scholars have long emphasised the importance of understanding institutional change and policy entrepreneurship. This review article is a response to this..."
TL;DR : "Political science and public policy scholars have long emphasised the importance of understanding institutional change and policy entrepreneurship."

Abstract : [mettez votre abstract]
TL;DR :"""